In [ ]:
# defining root location of the project on local system (change this to your own path after pulling)

ROOT = "/home/andreasstillits/coding/Leaves1D/"
PATH = ROOT + "figures/scripts/"

%cd $PATH

/home/andreasstillits/coding/Leaves1D/figures/scripts


'/home/andreasstillits/coding/Leaves1D/figures/scripts'

### Figure 2A

### Figure 2B

### Figure 2D

### Figure 2E

### Figure 2F